In [23]:
import sys, os
import numpy as np
import torch
from torch import Tensor
from torch.utils.data import Dataset
from torchvision import transforms
import nibabel as nib
from omegaconf import OmegaConf
import nrrd
import random

sys.path.append('..')
from user_model import UserModel
from utils import *




In [27]:
### Load basic config
cfg = OmegaConf.load('../configs/ae_training.yaml')

In [28]:
class PretrainingDataset(Dataset):
    
    def __init__(
        self, 
        cfg, 
    ):
        data_dir = cfg["data_dir"]
        subject_dirs = [
            d for d in os.listdir(data_dir) 
            if os.path.isdir(os.path.join(data_dir, d))
        ]
        if isinstance(cfg['subjects'], list):
            subject_dirs = [d for d in subject_dirs if d in cfg['subjects']]

            
        print(subject_dirs)

    #     data_in = torch.tensor(nib.load(cfg["data_path"]).get_fdata()).float()
        
    #     # shape [145, 288, 145, 145]  [B, C, H, W]
    #     if cfg['normalize']:
    #         self.input = (data_in / data_in.amax(dim=(0,1,2))).permute(1,3,0,2)[14:159]
    #     else:
    #         self.input = data_in.permute(1,3,0,2)#[14:159]
        
    #     # shape [145, 145, 145]       [B, H, W]
    #     self.brain_mask = torch.tensor(
    #         nib.load(cfg["active_mask_path"]).get_fdata(), dtype=torch.bool
    #     ).permute(1,0,2)#[14:159]
            
    #     #if cfg['log']:
    #     #    wandb.config.update({'labels': cfg['labels']})
        
    #     if cfg['to_gpu']:
    #         self.input      = self.input.to(self.cfg["rank"])
    #         self.brain_mask = self.brain_mask.to(self.cfg["rank"]) 

        
    # def __getitem__(self, index) -> dict:

    #     input_ = self.input[index]
    #     mask = self.brain_mask[index]
        
    #     return {
    #         'input': input_,
    #         'mask':  mask
    #     } 
    

    # def __len__(self) -> int:
    #     if self.balance:
    #         return self.index_tensor.shape[1]
    #     else:
    #         return self.input.shape[0]

In [29]:
test = PretrainingDataset(cfg)

['599469', '599671', '601127', '613538', '620434', '622236', '623844', '627549', '638049', '644044', '645551', '654754', '665254', '672756', '673455', '677968', '679568', '680957', '683256', '685058', '687163', '690152', '695768', '702133', '704238', '705341', '709551', '715041', '715647', '729254', '729557', '732243', '734045', '742549', '748258', '748662', '749361', '751348', '753251', '756055', '759869', '761957', '765056', '770352', '771354', '779370', '782561', '784565', '786569', '789373', '792564', '792766', '802844', '814649', '816653', '826353', '826454', '833148', '833249', '837560', '837964', '845458', '849971', '856766', '857263', '859671', '861456', '865363', '871762', '871964', '872158', '872764', '877168', '877269', '887373', '889579', '894673', '896778', '896879', '898176', '899885', '901038', '901139', '901442', '904044', '907656', '910241', '912447', '917255', '922854', '930449', '932554', '951457', '957974', '958976', '959574', '965367', '965771', '978578', '979984',